In [159]:
import tweepy
import requests
import os
from bing_image_downloader import downloader
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageStat
from pathlib import Path
import shutil

In [ ]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""

ACCESS_TOKEN = ""
ACCESS_SECRET = ""

In [161]:
# Sets up the Twitter API
def getAPI():
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)

    return api


# Populates an empty dictionary with tweets
def get_tweets(verbose):
    
    tweet_dict = {}  # initialize dictionary for tweets
    count = 0
    api = getAPI()
    # API.search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)
    dream_tweets = api.search_tweets('dream', count=10) # change back to 100 later, for testing just use small number
    for tweet in dream_tweets:
        if(len(tweet.user.location) != 0):
            tweet_dict[count] = {'location': tweet.user.location, 'id': tweet.id, 'profile_url': tweet.user.profile_image_url, \
                                 'text': tweet.text, 'geo': tweet.geo, 'source': tweet.source, 'created_at': tweet.created_at, \
                                'lang': tweet.lang }
            
            count += 1
    
    if verbose == True:
        print("number of tweets: "+ str(count))
    
    return tweet_dict

In [162]:
# Downloads satellite images from Bing of location from Tweet

def get_satellite_image(tweet):

    location = tweet['location']

    # note: if 'location' contains special characters, might throw error for creation of directory in dataset
    escaped = re.sub(r"[^a-zA-Z0-9 ]", "", location) # note: cleans out non-English
    location = escaped
    tweet['location'] = location # location string updated for referencing dir later
    
    query_string = location + ' satellite map'
    
    # check and clear old dataset if exists
    dirpath = Path('dataset') / str(query_string)
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)
    
    downloader.download(query_string, limit=3, output_dir='dataset', adult_filter_off=True, force_replace=False, timeout=60, verbose=False)
 
    return 

In [163]:
# Downloads twitter user's profile picture given tweet and count number 
def get_twitter_profile(tweet, count):    
    
    image_url = tweet['profile_url']
    img_data = requests.get(image_url).content
    
    # downloads to 'profile' directory
    while True:
        try:
            with open('./profile/' + str(count) + '.png', 'wb') as handler:
                handler.write(img_data)
            break
        except:
            print("Oops! Twitter profile img failed to download. ")
    
    return 1

# combines the satellite image and user profile associated with a tweet
def combine_images(tweet, count, isCrop): # boolean for 'isCrop'
        
    image = Image.open('./profile/' + str(count) + '.png') 
    image = image.resize((1000,1000))
    
    # image 2: open satellite map from folder
    query_string = tweet['location'] + ' satellite map'
    path = os.path.join("dataset", query_string) # folder path
    dir_list = os.listdir(path)
    path = os.path.join("dataset", query_string, dir_list[0]) # image 1 in folder path
    satelliteIm = Image.open(path)
    satelliteIm = satelliteIm.resize((1000,1000))
    
    if isCrop == True:
        image = crop_most_edges(image, False)
        satelliteIm = crop_most_edges(satelliteIm, False)    
 
   # convert images to ' ' mode
    image = image.convert('RGBA')
    satelliteIm = satelliteIm.convert('RGBA')
 
    # alpha is 0.5, 
    im3 = Image.blend(image, satelliteIm, 0.5)
 
    # return combined image
    return im3
        

In [164]:
# edge detection, crop based on which section has most edges
def crop_most_edges(image, verbose):

    # Converting the image to grayscale, as edge detection
    # requires input image to be of mode = Grayscale (L)
    image = image.convert("L")
    imageWithEdges = image.filter(ImageFilter.FIND_EDGES)

    # crop options
    left, top, right, bottom = 0, 0, 1000, 400 # top half
    crop_1 = imageWithEdges.crop((left, top, right, bottom))

    left, top, right, bottom = 0, 400, 1000, 800 # middle half
    crop_2 = imageWithEdges.crop((left, top, right, bottom))
    
    left, top, right, bottom = 0, 600, 1000, 1000 # bottom half
    crop_3 = imageWithEdges.crop((left, top, right, bottom))

    # calculate stats
    stat_1 = ImageStat.Stat(crop_1)
    stat_2 = ImageStat.Stat(crop_2)
    stat_3 = ImageStat.Stat(crop_3)
    stat = ImageStat.Stat(imageWithEdges)
    if verbose == True:
        print("crop_1: " + str(stat_1.mean))
        print("crop_2: " + str(stat_2.mean))
        print("crop_3: " + str(stat_3.mean))
    
    # decide which crop option to use
    option = max(stat_1.mean[0], stat_2.mean[0], stat_3.mean[0])
    if option == stat_2:
        image = image.crop((0, 400, 1000, 800)) # middle pt
    elif option == stat_1:
        image = image.crop((0, 0, 1000, 400)) # top pt
    else:
        image = image.crop((0, 600, 1000, 1000)) # bottom pt
    
    return image


In [165]:
#  Call draw Method to add 2D graphics in an image

def add_text(image, tweet):
    
    imageNew = ImageDraw.Draw(image)
    
    # Custom font style and font size
    myFont = ImageFont.truetype('simsun.ttc', 30)
    
    # Add Text to an image
    imageNew.text((10, 10), tweet['location'], font=myFont, fill =(0, 0, 255)) # text is location name
    imageNew.text((10, 50), tweet['text'], font=myFont, fill =(0, 0, 255))
    imageNew.text((10, 90), str(tweet['created_at']), font=myFont, fill =(0, 0, 255))

    return image

In [ ]:
# # Runs program on a single tweet
# inputs, # of tickets to print, # of tweets to collect

# populates dictionary with tweets
tweet_dict = get_tweets(True)

# downloads satellite image given one tweet
get_satellite_image(tweet_dict[0])

# downloads twitter user profile picture
get_twitter_profile(tweet_dict[0], 0)

# combines the two images
image = combine_images(tweet_dict[0], 0, True)

# give image path and tweet to add text 
image = add_text(image, tweet_dict[0])
image.show()

# converting to jpg
rgb_im = image.convert("RGB")
rgb_im.save("ticket.jpg")